In [4]:
import dspy
import os

from dotenv import load_dotenv

load_dotenv()  # 加载 .env 文件中的变量

True

In [5]:
# 设置语言模型
lm = dspy.LM('openai/qwen2.5-32b-instruct', api_base=os.getenv('QW_API_BASE'), api_key=os.getenv('QW_API_KEY'),
             cache=False)
dspy.configure(lm=lm)

## 示例1：情感分类

In [6]:
sentence = "it's a charming and often affecting journey."  # example from the SST-2 dataset.

classify = dspy.Predict('sentence -> sentiment: bool')  # we'll see an example with Literal[] later
classify(sentence=sentence).sentiment

True

## 示例2：摘要

In [9]:
# Example from the XSum dataset.
document = """我为什么要使用 DSPy 签名？用于模块化和干净的代码，其中 LM 调用可以优化为高质量的 prompts（或自动微调）。大多数人通过编写冗长、脆弱的 prompt 来强制 LM 执行任务。或者通过收集/生成数据进行微调。编写签名比手动修改 prompt 或微调更具模块化、适应性和可重复性。DSPy 编译器将根据您的签名、数据和管道，为您的 LM（或微调您的小型 LM）找出如何构建高度优化的 prompt。在许多情况下，我们发现编译生成的 prompt 比人工编写的更好。这并非因为 DSPy 优化器比人类更有创造力，而仅仅是因为它们可以尝试更多可能性并直接调整指标。"""

summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document=document)

print(response.summary)

DSPy签名用于模块化和干净的代码，使语言模型调用能够优化为高质量的prompts。相较于手动修改prompts或微调，DSPy更具模块化、适应性和可重复性。DSPy编译器能够根据签名、数据和管道找到优化的prompts，且通常优于人工编写的prompts，因为它能测试更多可能性并直接调整指标。


In [10]:
print("Reasoning:", response.reasoning)

Reasoning: 文档描述了DSPy签名的优点，包括模块化、适应性和可重复性，以及它如何帮助LM调优以生成高质量的prompts。与手动修改prompts或微调相比，DSPy能够通过其编译器找到更优化的prompts，这归功于其能够测试更多的可能性并直接调整指标的能力。因此，总结应强调这些关键点。


## 示例3：分类

In [12]:
from typing import Literal

class Emotion(dspy.Signature):
    """Classify emotion."""

    sentence: str = dspy.InputField()
    sentiment: Literal['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'] = dspy.OutputField()

sentence = "当那巨大的聚光灯开始刺得我睁不开眼时，我开始感到有点脆弱。"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='fear'
)

In [13]:
sentence = "今天天气真好~"

classify = dspy.Predict(Emotion)
classify(sentence=sentence)

Prediction(
    sentiment='joy'
)

## 示例4：评估引文忠实度的指标

In [15]:
class CheckCitationFaithfulness(dspy.Signature):
    """Verify that the text is based on the provided context."""

    context: str = dspy.InputField(desc="facts here are assumed to be true")
    text: str = dspy.InputField()
    faithfulness: bool = dspy.OutputField()
    evidence: dict[str, list[str]] = dspy.OutputField(desc="Supporting evidence for claims")

context = "这位 21 岁的球员为西汉姆联队出场 7 次，并在上赛季欧联杯资格赛对阵安道尔球队卢斯特雷恩斯的比赛中为球队打入唯一进球。去年，李先后被租借到英甲的布莱克浦和科尔切斯特联队。他在科尔切斯特联队打进两球，但未能帮助球队保级。李与升班马利兹联队签订的合同年限尚未公布。更多最新足球转会信息请访问我们的专题页面。"

text = "李为科尔切斯特联队贡献了 3 个进球。"

faithfulness = dspy.ChainOfThought(CheckCitationFaithfulness)
faithfulness(context=context, text=text)

Prediction(
    reasoning='文本声称李为科尔切斯特联队贡献了3个进球，但根据上下文，他实际上只为科尔切斯特联队打进了两个进球。因此，文本中的陈述不准确。',
    faithfulness=False,
    evidence={'李': ['他在科尔切斯特联队打进两球，但未能帮助球队保级。']}
)